# Total Cost of Ownership (TCO) Analysis

Last Updated on January 25, 2019

### *** To start off, select the first cell with Python code below and then press Shift + Enter to run the cell. You can continue to press Shift + Enter as the selection moves forward to the next cell. 

## Step 1: Initialize

In [ ]:
from IPython.display import display, HTML
from ipywidgets import widgets, interactive, Layout
import pandas as pd
import qgrid

import my_functions as my
import my_DB_functions as myDB

USER_ID = ! whoami
USER_ID = USER_ID[0]

records = myDB.get_avail_records(USER_ID)
df = pd.DataFrame(records, columns=["ID", "UserID", "UploadTime", "Org", "System", "Description", "Type", 
                                    "DeployDate", "Measure", "X", "Y", "BenchmarkDate",  "Availability"])
df = df.set_index("ID")
df.Availability = df.Availability.apply(lambda x:"Public" if x == 1 else "Private")

if len(df) == 0:
    print("No available data found.\nPlease upload your data first using manage_data.ipynb.")

## Step 2: Select records of interest

In [ ]:
print("Select one or more of the available records.\n"
      + "You can select multiple rows by pressing and holding the Ctrl key as you click a record.")
q = qgrid.show_grid(df.drop(["UploadTime"], axis=1))
display(q)

## Analyze TCO

In [ ]:
MAX_YEARS = 5

if len(q.get_selected_rows()) == 0:
    q.change_selection([1, 2, 3])

record_id_list = sorted(q.get_selected_df().index)
df_xy_list = []

for record_id in record_id_list:
    user_id = df["UserID"][record_id]
    ts = df["UploadTime"][record_id]
    org_name = df["Org"][record_id]
    sys_name = df["System"][record_id]
    sys_type = df["Type"][record_id]
    x = df["X"][record_id]
    y = df["Y"][record_id]

    records = myDB.get_selected_benchmark_records(user_id, ts)
    df_xy = pd.DataFrame(records, columns=["x", "y"])
    df_xy_list.append(df_xy)

###############################################################    
    
tco_widgets = {}

style = {"description_width": '300px'}
layout = Layout(width="45%")

for record_id in record_id_list:
    user_id = df["UserID"][record_id]
    ts = df["UploadTime"][record_id]
    org_name = df["Org"][record_id]
    sys_name = df["System"][record_id]
    sys_type = df["Type"][record_id]
        
    w_title = widgets.HTML(
        value = "<b>{} {} {}</b>".format(org_name, sys_name, sys_type),
        description = "ID {}: ".format(record_id),
        style = {"description_width": '50px'},
        layout = layout
    )
    tco_widgets["{} title".format(record_id)] = w_title
    
    cursor = myDB.get_tco_questions(sys_type)
    cursor2 = myDB.get_tco_data(sys_type, user_id, ts)
    
    tco_values = list(cursor2)[0]
    
    for item, tco_value in zip(list(cursor), tco_values[3:]):
        question = item[0]
        variable_name = item[1]
        unit = item[2]

        w = widgets.FloatText(
            description = "{} {}".format(question, "" if unit == "" else "({})".format(unit)),
            value = tco_value,
            layout = layout,
            style = style
        )
        
        tco_widgets["{} {}".format(record_id, variable_name)] = w

    w_degree = widgets.IntSlider(
        value = 2,
        min = 1,
        max = 3,
        step = 1,
        description = "Degree of Polynomial for Performance Scaling".format(record_id),
        layout = layout,
        style = style
    )

    tco_widgets["{} degree".format(record_id)] = w_degree

w_br = widgets.HTML(
    value = "<br>",
    description = " "
)
tco_widgets["br"] = w_br
    
style = {'description_width': '150px'}
layout = Layout(width="70%")

w_years = widgets.IntSlider(
    description = "Life Span (Years)",
    min = 0,
    max = MAX_YEARS,
    step = 1,
    value = 5,
    style = style,
    layout = layout
)
tco_widgets["years"] = w_years

w_util = widgets.IntSlider(
    description = "System Utilization (%)",
    min = 0,
    max = 100,
    step = 5,
    value = 100,
    style = style, 
    layout = layout
)
tco_widgets["util"] = w_util

def view(**tco_widgets):
    params_all = {}
    years = tco_widgets["years"]
    util = tco_widgets["util"]
    
    for key in tco_widgets:
        if key in ["years", "util", "br"]:
            continue
        
        record_id, variable_name = key.split()
        
        assert int(record_id) in record_id_list
        
        if record_id not in params_all:
            params_all[record_id] = dict()
            
        params_all[str(record_id)][variable_name] = tco_widgets[key]

    for record_id in record_id_list:
        params_all[str(record_id)]["org_name"] = df["Org"][record_id] 
        params_all[str(record_id)]["sys_name"] = df["System"][record_id]
        params_all[str(record_id)]["sys_type"] = df["Type"][record_id] 

    my.display_compare_TCO_new(years, util, df, record_id_list, df_xy_list, **params_all)

i = interactive(view, **tco_widgets)
display(i)